In [1]:
# Identificação facial

In [2]:
import os
import numpy as np
import cv2
import face_recognition

In [6]:
diretorio_trabalho = '/home/pi/Documents/Identificação facial/Indivíduos/'
os.chdir(diretorio_trabalho)

In [87]:
class Pessoa:
    def __init__(self, nome='Não definido'):
        self.nome = nome
        self.minuncias = []

    def calculaMinuncia(self, endereco_imagem):  
        imagem = face_recognition.load_image_file(endereco_imagem)
        minuncia = face_recognition.face_encodings(imagem)[0]
        self.minuncias.append(minuncia)     
    
    def imagemMatch(self, face_minuncias):
        dist = 0
        for aux_minuncias in self.minuncias:
            dist = dist + np.linalg.norm(face_minuncias-aux_minuncias)
        dist = dist/len(self.minuncias)      
        return dist

In [88]:
obama = Pessoa('Obama')

In [89]:
obama.calculaMinuncia(diretorio_trabalho+'Obama/obama.jpg')
obama.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.35.jpeg')

In [59]:
endereco_imagem = diretorio_trabalho+'Obama/obama.jpg'
imagem = face_recognition.load_image_file(endereco_imagem)
faces_locacoes = face_recognition.face_locations(imagem)
face_minuncias = None
n_faces = len(faces_locacoes)
if n_faces==1:
    print("Face detectada.")
    face_minuncias = face_recognition.face_encodings(imagem)[0]
elif n_faces==0:
    print("Nenhuma face detectada!")
else:
    print("Mais de uma face detectada!")       

Face detectada.


In [97]:
print(obama.imagemMatch(face_minuncias))

0.3513959518108559
